In [1]:
import soundfile as sf
import numpy as np
from IDMT_open import *
import os
import sys

parent_dir_path = os.path.dirname(os.getcwd())
sys.path.append(parent_dir_path)

from SoundMap.TimeDelay import *

In [2]:
id_of_last_long_record = 10

In [3]:
def combine_records_2min(index=id_of_last_long_record, location="Schleusinger-Allee", sr=48000):
    
    dataset_of_records = import_dataset_of_records("IDMT_traffic/IDMT_traffic/annotation/idmt_traffic_all.txt")
    dataset_of_records = dataset_of_records.loc[dataset_of_records['location']==location]
    
    txt_file_name = f"LongRecords/{index}.txt"
    
    labels = []
    final_signal_left, final_signal_right = np.array([]), np.array([])
    last_index=0
    for file in dataset_of_records.iloc[index*60:(index+1)*60].iloc:
        signal = import_record(file['file'])
        time_delay = time_delay_2d(signal)
        if file['is_background']:
            labels.append(f"N|{last_index},{len(time_delay)+last_index}")
        else:
            labels.append(f"{file['source_direction']}|{last_index},{len(time_delay)+last_index}")
        last_index += len(time_delay) + 7
        final_signal_left, final_signal_right = np.append(final_signal_left, signal[0]), np.append(final_signal_right, signal[1])
    
    final_signal = np.array([final_signal_left, final_signal_right]).T
    record_name = f"LongRecords/{index}.wav"
    
    sf.write(record_name, final_signal, sr)
    
    txt_long_record = open(txt_file_name, "w")
    txt_long_record.write("_".join(labels))
    txt_long_record.close()
    

In [4]:
combine_records_2min(id_of_last_long_record)